In [3]:
import pandas as pd 
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

df=pd.read_csv("covid_toy.csv")
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

# X_train.head()
y_train.head()

62    Yes
18    Yes
45     No
44     No
75    Yes
Name: has_covid, dtype: object

In [12]:
# //Adding Simple Imputer
si=SimpleImputer()
X_train_fever=si.fit_transform(X_train[['fever']])
X_test_fever=si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [13]:
oe=OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough=oe.fit_transform(X_train[['cough']])
X_test_cough=oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [16]:
# One Hot Encoding
ohe=OneHotEncoder(drop='first',sparse_output=False)

X_train_gender_city=ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city=ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [18]:
X_train_age=X_train.drop(columns=['gender','fever','cough','city']).values
X_test_age=X_test.drop(columns=['gender','fever','cough','city']).values

X_test_age.shape

(20, 1)

In [26]:
X_train_transformed=np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

X_train_transformed
data=pd.DataFrame(X_train_transformed)
data.head()

,0,1,2,3,4,5,6
0,56.0,104.0,0.0,0.0,0.0,0.0,1.0
1,64.0,98.0,0.0,0.0,0.0,0.0,0.0
2,72.0,99.0,1.0,0.0,0.0,0.0,0.0
3,20.0,102.0,1.0,1.0,0.0,0.0,1.0
4,5.0,102.0,1.0,0.0,1.0,0.0,0.0


In [37]:
# // Mentos Jindagi

from sklearn.compose import ColumnTransformer

transformer=ColumnTransformer(transformers=[
    ('t1',SimpleImputer(),['fever']),
    ('t2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('t3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

X_train_transformed_ct=transformer.fit_transform(X_train)

ct_columns=['fever']+['cough']+ohe.get_feature_names_out(['gender','city']).tolist()+['age']

ct_data=pd.DataFrame(X_train_transformed_ct,columns=ct_columns)
ct_data.head()

,fever,cough,gender_Male,city_Delhi,city_Kolkata,city_Mumbai,age
0,104.0,1.0,0.0,0.0,0.0,0.0,56.0
1,98.0,0.0,0.0,0.0,0.0,0.0,64.0
2,99.0,0.0,1.0,0.0,0.0,0.0,72.0
3,102.0,1.0,1.0,1.0,0.0,0.0,20.0
4,102.0,0.0,1.0,0.0,1.0,0.0,5.0
